In [1]:
# Install a conda package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install flickrapi


In [2]:
import flickrapi
import pandas as pd
#import sys



In [4]:
#set "key" and "secret" in a local configuration file named flickr_APIkey.txt
#for an example see: flickr_APIkey_example.txt
#read-in key and secret from configuration file
import os
os.chdir("/Users/KGthatsme/Analyses/FlickrFish")
file=open(file="RESULTS/flickr_APIkey.txt", mode="r") #mode r = read the file
key=file.readline()
key=key.rstrip() # Remove all trailing whitespace (e.g., \n)
secret=file.readline()
secret=secret.rstrip()

In [5]:
## Example of how to print json output of photo search
image_tag="fish"
MAX_COUNT=5
flickr = flickrapi.FlickrAPI(key, secret, format='parsed-json')
fishes = flickr.photos.search(#text=image_tag, #use text parameter instead, which searches title, description, and tags
                         #tag_mode='all',
                         tags=image_tag, 
                         #bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='url_o, date_upload, date_taken, geo, machine_tags', # still not sure what machine_tags is for
                         per_page=MAX_COUNT,
                         sort='relevance'
                        )
photos = fishes['photos']
from pprint import pprint
pprint(photos)

{'page': 1,
 'pages': 72207,
 'perpage': 5,
 'photo': [{'accuracy': '16',
            'context': 0,
            'datetaken': '2019-10-05 16:05:15',
            'datetakengranularity': '0',
            'datetakenunknown': '0',
            'dateupload': '1570785027',
            'farm': 66,
            'geo_is_contact': 0,
            'geo_is_family': 0,
            'geo_is_friend': 0,
            'geo_is_public': 1,
            'id': '48879645352',
            'isfamily': 0,
            'isfriend': 0,
            'ispublic': 1,
            'latitude': '52.659711',
            'longitude': '-0.691181',
            'machine_tags': '',
            'owner': '145783492@N07',
            'place_id': '',
            'secret': '3dc6bc5708',
            'server': '65535',
            'title': "I've told you lot before - Leave the fish alone!",
            'woeid': '2635127'},
           {'accuracy': '16',
            'context': 0,
            'datetaken': '2019-09-05 10:25:09',
            'date

In [6]:
def get_urls(image_tag, MAX_COUNT):    
    flickr = flickrapi.FlickrAPI(key, secret)
    count=0
    urls=[]
    uploads=[]
    lats=[]
    longs=[]
    takens=[]
    accuracies=[]
    photos = flickr.walk(text=image_tag, #use text parameter instead, which searches title, description, and tags
                         #tags=image_tag, 
                         bbox='-71.791037, 41.217233, -71.095148, 42.007849', # Narraganset Bay + RI Sound
                         has_geo=1,
                         #lat=41.604168,
                         #lon=-71.320683,
                         extras='url_t, date_upload, date_taken, geo',
                         # use url_t instead of url_o; more search results provide access to "thumbnail" photo than the "original" photo
                         per_page=MAX_COUNT, 
                         sort='relevance'
                        )
    for photo in photos:
        if count<MAX_COUNT:
            count=count+1
            print("Fetching data for image number {}".format(count))
            try:
                url=photo.get('url_t') 
                print("URL: {}".format(url))
                urls.append(url)
                upload=photo.get('dateupload')
                print("Upload: {}".format(upload))
                uploads.append(upload)
                taken=photo.get('datetaken')
                print("Taken: {}".format(taken))
                takens.append(taken)
                lat=photo.get('latitude')
                print("Latitude: {}".format(lat))
                lats.append(lat)
                long=photo.get('longitude')
                print("Longitude: {}".format(lat))
                longs.append(long)
                accuracy=photo.get('accuracy')
                print("Accuracy: {}".format(accuracy)) # NOTE: may have to filter results by accuracy (16 = STREET LEVEL)
                accuracies.append(accuracy)
            except:
                print("Url for image number {} could not be fetched".format(count))
        else:
            print("Done fetching urls, fetched {} urls out of {}".format(len(urls),MAX_COUNT))
            break
    urls=pd.Series(urls)
    uploads=pd.Series(uploads)
    lats=pd.Series(lats)
    longs=pd.Series(longs)
    takens=pd.Series(takens)
    accuracies=pd.Series(accuracies)
    print("Writing data to the current directory")
    urls.to_csv(image_tag+"_urls.csv")
    uploads.to_csv(image_tag+"_uploads.csv")
    takens.to_csv(image_tag+"_takens.csv")
    lats.to_csv(image_tag+"_lats.csv")
    longs.to_csv(image_tag+"_longs.csv")
    accuracies.to_csv(image_tag+"_accuracies.csv")
    print("Done!!!")



In [7]:
# change wd to RESULTS folder
os.chdir("/Users/KGthatsme/Analyses/FlickrFish/RESULTS")

In [8]:
image_tag='fishing'
MAX_COUNT=1000 # search results max out around 820
get_urls(image_tag=image_tag, MAX_COUNT=MAX_COUNT)

Fetching data for image number 1
URL: https://live.staticflickr.com/5172/5535174301_bcdb80b09f_t.jpg
Upload: 1300394978
Taken: 2011-03-17 16:49:38
Latitude: 41.325473
Longitude: 41.325473
Accuracy: 15
Fetching data for image number 2
URL: https://live.staticflickr.com/3007/2970477836_cb4af4d8ed_t.jpg
Upload: 1224887536
Taken: 2008-10-24 07:35:50
Latitude: 41.454049
Longitude: 41.454049
Accuracy: 16
Fetching data for image number 3
URL: https://live.staticflickr.com/2922/14430114968_5975944976_t.jpg
Upload: 1404957494
Taken: 2014-07-08 19:51:12
Latitude: 41.682033
Longitude: 41.682033
Accuracy: 16
Fetching data for image number 4
URL: https://live.staticflickr.com/8305/7824793148_ea838e260d_t.jpg
Upload: 1345482595
Taken: 2012-08-14 13:34:33
Latitude: 41.378741
Longitude: 41.378741
Accuracy: 16
Fetching data for image number 5
URL: https://live.staticflickr.com/3845/14775543106_815e14fc15_t.jpg
Upload: 1406858620
Taken: 2014-07-08 19:40:05
Latitude: 41.682027
Longitude: 41.682027
Accura

Fetching data for image number 251
URL: https://live.staticflickr.com/7885/47486249261_d5db93b996_t.jpg
Upload: 1553784692
Taken: 2019-03-28 10:33:46
Latitude: 41.569594
Longitude: 41.569594
Accuracy: 16
Fetching data for image number 252
URL: https://live.staticflickr.com/7811/40358799813_9377379d95_t.jpg
Upload: 1552103551
Taken: 2019-02-22 15:43:05
Latitude: 41.570813
Longitude: 41.570813
Accuracy: 16
Fetching data for image number 253
URL: https://live.staticflickr.com/914/43041821721_accaeddf27_t.jpg
Upload: 1530102910
Taken: 2018-06-27 08:31:04
Latitude: 41.947066
Longitude: 41.947066
Accuracy: 16
Fetching data for image number 254
URL: https://live.staticflickr.com/7473/15953223008_e160f77a73_t.jpg
Upload: 1419884233
Taken: 2014-06-01 00:00:00
Latitude: 41.918930
Longitude: 41.918930
Accuracy: 15
Fetching data for image number 255
URL: https://live.staticflickr.com/6151/6235372152_4d44e4a6df_t.jpg
Upload: 1318359760
Taken: 2011-10-05 15:31:50
Latitude: 41.412128
Longitude: 41.41

Fetching data for image number 501
URL: https://live.staticflickr.com/5562/14548569440_40248f7c7d_t.jpg
Upload: 1406225377
Taken: 2014-07-06 15:42:58
Latitude: 41.583277
Longitude: 41.583277
Accuracy: 16
Fetching data for image number 502
URL: https://live.staticflickr.com/4143/4903156326_a2396aaef5_t.jpg
Upload: 1282092727
Taken: 2010-08-14 11:13:24
Latitude: 41.449678
Longitude: 41.449678
Accuracy: 16
Fetching data for image number 503
URL: https://live.staticflickr.com/4076/4903145756_39a45a1e45_t.jpg
Upload: 1282092451
Taken: 2010-08-14 10:38:19
Latitude: 41.449678
Longitude: 41.449678
Accuracy: 16
Fetching data for image number 504
URL: https://live.staticflickr.com/7774/18006398059_85a3924169_t.jpg
Upload: 1432810502
Taken: 2015-05-28 06:40:00
Latitude: 41.946988
Longitude: 41.946988
Accuracy: 16
Fetching data for image number 505
URL: https://live.staticflickr.com/7907/32996421348_31c813bb47_t.jpg
Upload: 1548425504
Taken: 2019-01-25 07:59:31
Latitude: 41.947122
Longitude: 41.94

Fetching data for image number 751
URL: https://live.staticflickr.com/8661/29593304115_3e39984735_t.jpg
Upload: 1473549425
Taken: 2016-09-10 16:17:05
Latitude: 41.706726
Longitude: 41.706726
Accuracy: 16
Fetching data for image number 752
URL: https://live.staticflickr.com/3762/20171396612_f0a3f71fc8_t.jpg
Upload: 1438374338
Taken: 2015-07-31 13:46:41
Latitude: 41.555947
Longitude: 41.555947
Accuracy: 16
Fetching data for image number 753
URL: https://live.staticflickr.com/65535/48907798403_ff7a628638_t.jpg
Upload: 1571227111
Taken: 2019-10-16 07:29:22
Latitude: 41.947008
Longitude: 41.947008
Accuracy: 16
Fetching data for image number 754
URL: https://live.staticflickr.com/65535/48743132126_949a7fc2c0_t.jpg
Upload: 1568638023
Taken: 2019-09-16 08:01:34
Latitude: 41.947436
Longitude: 41.947436
Accuracy: 16
Fetching data for image number 755
URL: https://live.staticflickr.com/65535/48519199436_33db3509fd_t.jpg
Upload: 1565612890
Taken: 2019-08-10 14:05:49
Latitude: 41.553417
Longitude: 

In [9]:
## Only need the following code if running this from Terminal
#def main(): 
#    tag=sys.argv[1]
#    MAX_COUNT=int(sys.argv[2])
#    get_urls(tag,MAX_COUNT)


#if __name__=='__main__': 
#    main() 
# if running the module as the main program (e.g., in terminal: python getFishDat.py): execute function (main)
# if module is imported by another program, _name_ is assigned to the module name "getFishDat": skip function (main)

import csv
import requests
import os
import sys
import time
def put_images(image_tag):
    urls=[]
    FILE_NAME=image_tag+"_urls.csv"
    with open(FILE_NAME,newline="") as csvfile:
        doc=csv.reader(csvfile,delimiter=",")
        for row in doc:
            if row[1].startswith("https"):
                urls.append(row[1])
    if not os.path.isdir(os.path.join(os.getcwd(),FILE_NAME.split("_")[0])):
        os.mkdir(FILE_NAME.split("_")[0])
    t0=time.time()
    for url in enumerate(urls):
        print("Starting download {} of ".format(url[0]+1),len(urls))
        try:
            resp=requests.get(url[1],stream=True)
            path_to_write=os.path.join(os.getcwd(),FILE_NAME.split("_")[0],url[1].split("/")[-1])
            outfile=open(path_to_write,'wb')
            outfile.write(resp.content)
            outfile.close()
            print("Done downloading {} of {}".format(url[0]+1,len(urls)))
        except:
            print("Failed to download url number {}".format(url[0]))
    t1=time.time()
    print("Done with download, job took {} seconds".format(t1-t0))





In [10]:
put_images(image_tag=image_tag)

Starting download 1 of  823
Done downloading 1 of 823
Starting download 2 of  823
Done downloading 2 of 823
Starting download 3 of  823
Done downloading 3 of 823
Starting download 4 of  823
Done downloading 4 of 823
Starting download 5 of  823
Done downloading 5 of 823
Starting download 6 of  823
Done downloading 6 of 823
Starting download 7 of  823
Done downloading 7 of 823
Starting download 8 of  823
Done downloading 8 of 823
Starting download 9 of  823
Done downloading 9 of 823
Starting download 10 of  823
Done downloading 10 of 823
Starting download 11 of  823
Done downloading 11 of 823
Starting download 12 of  823
Done downloading 12 of 823
Starting download 13 of  823
Done downloading 13 of 823
Starting download 14 of  823
Done downloading 14 of 823
Starting download 15 of  823
Done downloading 15 of 823
Starting download 16 of  823
Done downloading 16 of 823
Starting download 17 of  823
Done downloading 17 of 823
Starting download 18 of  823
Done downloading 18 of 823
Starting d

Done downloading 147 of 823
Starting download 148 of  823
Done downloading 148 of 823
Starting download 149 of  823
Done downloading 149 of 823
Starting download 150 of  823
Done downloading 150 of 823
Starting download 151 of  823
Done downloading 151 of 823
Starting download 152 of  823
Done downloading 152 of 823
Starting download 153 of  823
Done downloading 153 of 823
Starting download 154 of  823
Done downloading 154 of 823
Starting download 155 of  823
Done downloading 155 of 823
Starting download 156 of  823
Done downloading 156 of 823
Starting download 157 of  823
Done downloading 157 of 823
Starting download 158 of  823
Done downloading 158 of 823
Starting download 159 of  823
Done downloading 159 of 823
Starting download 160 of  823
Done downloading 160 of 823
Starting download 161 of  823
Done downloading 161 of 823
Starting download 162 of  823
Done downloading 162 of 823
Starting download 163 of  823
Done downloading 163 of 823
Starting download 164 of  823
Done downloadi

Done downloading 289 of 823
Starting download 290 of  823
Done downloading 290 of 823
Starting download 291 of  823
Done downloading 291 of 823
Starting download 292 of  823
Done downloading 292 of 823
Starting download 293 of  823
Done downloading 293 of 823
Starting download 294 of  823
Done downloading 294 of 823
Starting download 295 of  823
Done downloading 295 of 823
Starting download 296 of  823
Done downloading 296 of 823
Starting download 297 of  823
Done downloading 297 of 823
Starting download 298 of  823
Done downloading 298 of 823
Starting download 299 of  823
Done downloading 299 of 823
Starting download 300 of  823
Done downloading 300 of 823
Starting download 301 of  823
Done downloading 301 of 823
Starting download 302 of  823
Done downloading 302 of 823
Starting download 303 of  823
Done downloading 303 of 823
Starting download 304 of  823
Done downloading 304 of 823
Starting download 305 of  823
Done downloading 305 of 823
Starting download 306 of  823
Done downloadi

Done downloading 432 of 823
Starting download 433 of  823
Done downloading 433 of 823
Starting download 434 of  823
Done downloading 434 of 823
Starting download 435 of  823
Done downloading 435 of 823
Starting download 436 of  823
Done downloading 436 of 823
Starting download 437 of  823
Done downloading 437 of 823
Starting download 438 of  823
Done downloading 438 of 823
Starting download 439 of  823
Done downloading 439 of 823
Starting download 440 of  823
Done downloading 440 of 823
Starting download 441 of  823
Done downloading 441 of 823
Starting download 442 of  823
Done downloading 442 of 823
Starting download 443 of  823
Done downloading 443 of 823
Starting download 444 of  823
Done downloading 444 of 823
Starting download 445 of  823
Done downloading 445 of 823
Starting download 446 of  823
Done downloading 446 of 823
Starting download 447 of  823
Done downloading 447 of 823
Starting download 448 of  823
Done downloading 448 of 823
Starting download 449 of  823
Done downloadi

Done downloading 575 of 823
Starting download 576 of  823
Done downloading 576 of 823
Starting download 577 of  823
Done downloading 577 of 823
Starting download 578 of  823
Done downloading 578 of 823
Starting download 579 of  823
Done downloading 579 of 823
Starting download 580 of  823
Done downloading 580 of 823
Starting download 581 of  823
Done downloading 581 of 823
Starting download 582 of  823
Done downloading 582 of 823
Starting download 583 of  823
Done downloading 583 of 823
Starting download 584 of  823
Done downloading 584 of 823
Starting download 585 of  823
Done downloading 585 of 823
Starting download 586 of  823
Done downloading 586 of 823
Starting download 587 of  823
Done downloading 587 of 823
Starting download 588 of  823
Done downloading 588 of 823
Starting download 589 of  823
Done downloading 589 of 823
Starting download 590 of  823
Done downloading 590 of 823
Starting download 591 of  823
Done downloading 591 of 823
Starting download 592 of  823
Done downloadi

Done downloading 718 of 823
Starting download 719 of  823
Done downloading 719 of 823
Starting download 720 of  823
Done downloading 720 of 823
Starting download 721 of  823
Done downloading 721 of 823
Starting download 722 of  823
Done downloading 722 of 823
Starting download 723 of  823
Done downloading 723 of 823
Starting download 724 of  823
Done downloading 724 of 823
Starting download 725 of  823
Done downloading 725 of 823
Starting download 726 of  823
Done downloading 726 of 823
Starting download 727 of  823
Done downloading 727 of 823
Starting download 728 of  823
Done downloading 728 of 823
Starting download 729 of  823
Done downloading 729 of 823
Starting download 730 of  823
Done downloading 730 of 823
Starting download 731 of  823
Done downloading 731 of 823
Starting download 732 of  823
Done downloading 732 of 823
Starting download 733 of  823
Done downloading 733 of 823
Starting download 734 of  823
Done downloading 734 of 823
Starting download 735 of  823
Done downloadi

In [ ]:
# CREATE MAP OF QUERY LAT/LONG
